In [1]:
import pandas as pd
import urllib,json
from pandas.io.json import json_normalize
import os
import inquirer
from datetime import datetime
from sklearn import preprocessing
from PySAM.PySSC import PySSC

In [2]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
from time import strptime
import calendar

In [3]:
def API_Reader(url):
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    json_load = json.loads(response.read())
    return(json_load)

In [4]:
eia_api_key = 'b0cbfe0b334c7c98b857879d54226b4a'
ei_api_key = 'o8dCYVBSkm1c2tnB5YUAYsKv4iZaC8AkyPYMYSBL'

# Rate Pull

In [5]:
Pay_Structure_DF = pd.DataFrame()
y = range(0,100)
for x in y:
    print(x),
    if x == 0:
        offset=0
    if x ==1:
        offset=501
    if x>1:
        offset=501 +((x-1)*500)
    api_type = 'csv'
    EI_url = 'https://api.openei.org/utility_rates?version=latest&format='+api_type+'&limit=500&sector=Industrial&detail=full&offset='+str(offset)+'&api_key=' + ei_api_key
    EI_API_CSV = pd.read_csv(EI_url)
    Pay_Structure_DF = pd.concat([Pay_Structure_DF,EI_API_CSV], axis=0, join='outer', ignore_index=True, keys=None,levels=None, names=None, verify_integrity=False, copy=True)
    if EI_API_CSV.shape[0] < 500:
            break
Pay_Structure_DF.to_csv('Pay_Structure.csv')

0
1


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [6]:
base_columns = ['label','uri',
    'approved',
    'is_default',
    'name',
    'startdate',
    'enddate',
    'utility',
    'eiaid',
    'country',
    'sector',
    'servicetype',
    'description',
    'source',
    'sourceparent',
    'revisions']
coincident_columns = list(Pay_Structure_DF.columns[Pay_Structure_DF.columns.str.contains('coincident')])# + ['coincidentratestructure/period2/tier0rate']
needed_columns = base_columns + coincident_columns
Coincident_DF = Pay_Structure_DF.loc[Pay_Structure_DF['coincidentrateunit'][Pay_Structure_DF['coincidentrateunit'].notna()].index,needed_columns ]
Coincident_DF.reset_index(inplace = True,drop = True)

In [7]:
coincidentschedulecolumns = EI_API_CSV.columns[EI_API_CSV.columns.str.contains('coincidentschedule')]
id_columns = [x for x in needed_columns if x not in coincidentschedulecolumns]
Coincident_DF_Schedule = pd.melt(Coincident_DF,id_vars = id_columns,value_vars = coincidentschedulecolumns,var_name = 'month_hour',value_name = 'schedule')

In [8]:
Coincident_DF_Schedule['month'] = Coincident_DF_Schedule.month_hour.str.split('_').str[1]
Coincident_DF_Schedule['hour'] = Coincident_DF_Schedule.month_hour.str.split('_').str[2]
d= {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9,
       'oct':10, 'nov':11, 'dec':12}
Coincident_DF_Schedule['month'].replace(d,inplace = True)

In [9]:
Coincident_DF_Schedule.loc[Coincident_DF_Schedule['schedule'] == 0,'price'] = Coincident_DF_Schedule['coincidentratestructure/period0/tier0rate']
Coincident_DF_Schedule.loc[Coincident_DF_Schedule['schedule'] == 1,'price'] = Coincident_DF_Schedule['coincidentratestructure/period1/tier0rate']
Coincident_DF_Schedule.loc[Coincident_DF_Schedule['schedule'] == 2,'price'] = Coincident_DF_Schedule['coincidentratestructure/period2/tier0rate']
Coincident_DF_Schedule.hour = Coincident_DF_Schedule.hour.astype(int)
Coincident_DF_Schedule.month = Coincident_DF_Schedule.month.astype(int)
Coincident_DF_Schedule.sort_values(by = ['utility','month','hour'],inplace = True)
Coincident_DF_Schedule.reset_index(inplace = True,drop = True)
CP_Utilities = Coincident_DF.eiaid.unique()

In [33]:
len(Coincident_DF_Schedule.utility.unique())
Coincident_DF_Schedule['utility'].unique()

array(['Access Energy Coop', 'Agralite Electric Coop',
       'Bangor Hydro-Electric Co', 'Bon Homme Yankton El Assn, Inc',
       'Butler County Rural Elec Coop', 'C & L Electric Coop Corp',
       'Carroll County REMC', 'Chippewa Valley Electric Coop',
       'City of Clintonville, Wisconsin (Utility Company)',
       'City of Fort Collins, Colorado (Utility Company)',
       'City of Gonzales, Texas (Utility Company)',
       'City of Lexington, North Carolina (Utility Company)',
       'City of Memphis, Tennessee (Utility Company)',
       'City of Monroe, North Carolina (Utility Company)',
       'City of Murfreesboro, Tennessee (Utility Company)',
       'City of Rocky Mount, North Carolina (Utility Company)',
       'City of Stanton, Iowa (Utility Company)',
       'City of Thief River Falls, Minnesota (Utility Company)',
       'City of Volga, South Dakota (Utility Company)',
       'City of Webster City, Iowa (Utility Company)',
       'Clay County Electric Coop Corp', 'Coast 

# Utility Rules

In [11]:
Utitility_Rules = pd.read_csv('https://raw.githubusercontent.com/awendell989/EIA_2018_Utiltiy_Policies/master/Dynamic_Pricing_2018_Avery.csv')
Utitility_Rules.loc[:,Utitility_Rules.columns[Utitility_Rules.columns.str.contains('--')]] = Utitility_Rules.loc[:,Utitility_Rules.columns[Utitility_Rules.columns.str.contains('--')]].replace({'N':False,'Y':True})
Industrial_Columns = Utitility_Rules.select_dtypes(include='bool').columns
Industrial_Columns = Industrial_Columns[Industrial_Columns.str.contains('Industrial')]
Utitility_Rules['Industrial_Total'] = Utitility_Rules[Industrial_Columns].sum(axis = 1)
CP_Options = Industrial_Columns + ['Industrial_Total']

In [12]:
#Coincident_DF = Coincident_DF.merge(Utitility_Rules[['Utility Number','BA Code']],left_on = 'eiaid',right_on = 'Utility Number')
Coincident_DF_Schedule_Full = Coincident_DF_Schedule.merge(Utitility_Rules[['Utility Number','BA Code']],left_on = 'eiaid',right_on = 'Utility Number')
CP_BAs = Coincident_DF_Schedule_Full['BA Code'].unique()

In [13]:
len(Coincident_DF_Schedule_Full.eiaid.unique())

20

# Demand Data

In [14]:
demand_total_url = 'http://api.eia.gov/category/?api_key=b0cbfe0b334c7c98b857879d54226b4a&category_id=2122628'
json_load = API_Reader(demand_total_url)
Categories = pd.DataFrame(json_load['category']['childcategories'])
Categories['BA_Code'] = Categories['name'].apply(lambda r: r.split('(')[1].split(')')[0])

In [15]:
Categories = Categories.loc[Categories['BA_Code'].isin(CP_BAs),:].reset_index(drop = True)
Categories

,category_id,name,BA_Code
0,3389963,Duke Energy Progress East (CPLE),CPLE
1,3389980,"Midcontinent Independent System Operator, Inc....",MISO
2,3389988,"PJM Interconnection, LLC (PJM)",PJM
3,3389994,Public Service Company of Colorado (PSCO),PSCO
4,3390005,"Southern Company Services, Inc. - Trans (SOCO)",SOCO
5,3390006,Southwest Power Pool (SWPP),SWPP
6,3390009,Tennessee Valley Authority (TVA),TVA
7,3390014,Western Area Power Administration - Rocky Moun...,WACM


In [16]:
Demand_Data_Total = pd.DataFrame(columns = ['UTC', 'MWh', 'BA', 'CID'])
for x in range(0,max(Categories.index+1)):
    name = Categories.iloc[x,1]
    category_id = Categories.iloc[x,0]
    category_url ='http://api.eia.gov/category/?api_key='+ str(eia_api_key) +'&category_id='+str(category_id)
    BA_Data = API_Reader(category_url)
    Series = pd.DataFrame(BA_Data['category']['childseries'])
    Series_id = Series.loc[Series['name'].str.contains('UTC'),'series_id'][0]
    series_url = 'https://api.eia.gov/series/?api_key='+eia_api_key+'&series_id='+str(Series_id)
    Series_Data = API_Reader(series_url)
    step1 = Series_Data['series'][0]
    step2 = step1['data']
    Columns = ['UTC','MWh']
    DF = pd.DataFrame(step2,columns = Columns)
    DF['BA'] = name
    DF['CID'] = category_id
    Demand_Data_Total = pd.concat([Demand_Data_Total,DF], axis=0, join='outer', ignore_index=True, keys=None,levels=None, names=None, verify_integrity=False, copy=True)
    print(name + ' added!')
    Demand_Data_Total['UTC'] = pd.to_datetime(Demand_Data_Total['UTC'],utc=True) #Is this True DOh?
    Demand_Data_Total['BA Code'] = Demand_Data_Total['BA'].apply(lambda r: r.split('(')[1].split(')')[0])

Duke Energy Progress East (CPLE) added!


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Midcontinent Independent System Operator, Inc. (MISO) added!
PJM Interconnection, LLC (PJM) added!
Public Service Company of Colorado (PSCO) added!
Southern Company Services, Inc. - Trans (SOCO) added!
Southwest Power Pool (SWPP) added!
Tennessee Valley Authority (TVA) added!
Western Area Power Administration - Rocky Mountain Region (WACM) added!


In [17]:
Demand_Data_Total['hour'] = Demand_Data_Total['UTC'].dt.hour
Demand_Data_Total['hour_p'] = Demand_Data_Total['UTC'].dt.hour/24
Demand_Data_Total['month'] = Demand_Data_Total['UTC'].dt.month
Demand_Data_Total['year'] = Demand_Data_Total['UTC'].dt.year
Demand_Data_Total['day'] = Demand_Data_Total['UTC'].dt.day
Demand_Data_Total['Rank'] = Demand_Data_Total.groupby(['BA','month','year'],axis = 0)['MWh'].rank(method = 'max',ascending = False)
Demand_Data_Total['Day_Hour'] = Demand_Data_Total['day'] + Demand_Data_Total['hour_p']
current_month = datetime.now().month
current_year = datetime.now().year
Demand_Data_Total = Demand_Data_Total[~((Demand_Data_Total['month'] == current_month) & (Demand_Data_Total['year'] == current_year))]
Demand_Data_Total = Demand_Data_Total.loc[Demand_Data_Total['BA Code'].isin(CP_BAs),:]

In [18]:
Month_Min_Max = Demand_Data_Total.groupby(['BA','year','month'],axis = 0).agg({'MWh' : ['min','max']}).reset_index()
Demand_Data_Total = Demand_Data_Total.merge(Month_Min_Max,on =['BA','year','month'])
Demand_Data_Total.rename(columns = {Demand_Data_Total.columns[-1] : 'm_max',Demand_Data_Total.columns[-2] : 'm_min'},inplace = True)
Demand_Data_Total['MWH_M_Scale'] = Demand_Data_Total.apply(lambda x: (x['MWh'] - x['m_min']) / (x['m_max'] - x['m_min']),axis = 1)

/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


In [19]:
Demand_Data_Total = Demand_Data_Total.loc[~((Demand_Data_Total['month'] == 2) & (Demand_Data_Total['day'] == 29)) ,]
Time_DF = Demand_Data_Total.groupby(['month','day','hour']).size().reset_index().drop(0,axis =1)

In [20]:
Demand_Data_Total.to_csv('Demand_by_BA.csv')

In [21]:
#Demand_Data_Total_Agg = Demand_Data_Total.groupby(['BA', 'BA Code', 'CID','month','day','hour','Day_Hour'])['MWH_M_Scale','MWh'].sum()
Demand_Data_Total_Agg = Demand_Data_Total.groupby(['BA', 'BA Code', 'CID','month','hour'])['MWH_M_Scale','MWh'].mean()
Demand_Data_Total_Agg.reset_index(inplace = True)
Demand_Data_Total_Agg.head()

,BA,BA Code,CID,month,hour,MWH_M_Scale,MWh
0,Duke Energy Progress East (CPLE),CPLE,3389963,1,0,0.437122,8257.548387
1,Duke Energy Progress East (CPLE),CPLE,3389963,1,1,0.442398,8304.362903
2,Duke Energy Progress East (CPLE),CPLE,3389963,1,2,0.438212,8268.612903
3,Duke Energy Progress East (CPLE),CPLE,3389963,1,3,0.415773,8074.056452
4,Duke Energy Progress East (CPLE),CPLE,3389963,1,4,0.378684,7752.233871


# Merge to Rate Schedule

In [22]:
Demand_Rate_Schedule = Demand_Data_Total_Agg.merge(Coincident_DF_Schedule_Full, how='right', left_on=['month','hour','BA Code'], right_on=['month','hour','BA Code'])
Demand_Rate_Schedule = Demand_Rate_Schedule[['BA Code','utility','eiaid','name','month','hour','MWh','MWH_M_Scale','schedule','price']].sort_values(by =['BA Code','utility','eiaid','name','month','hour'] ).reset_index(drop = True)

In [23]:
Demand_Rate_Schedule.to_csv('Demand_Rate_Schedule.csv',index = False)

In [24]:
len(Demand_Rate_Schedule.eiaid.unique())

20

# Zipcodes

In [25]:
U_to_Z_I = pd.read_csv('iouzipcodes2011.csv',dtype = {'zip': str})
U_to_Z_NI = pd.read_csv('noniouzipcodes2011.csv',dtype = {'zip': str})

In [27]:
#Concat And Run

U_to_Z = pd.concat([U_to_Z_I,U_to_Z_NI])
U_to_Z = U_to_Z.drop_duplicates(subset=['zip','eiaid'],keep = 'last').sort_values(by = 'zip')
#U_to_Z_Zips = U_to_Z.loc[U_to_Z['eiaid'].isin(CP_Utilities),'zip']
file = os.path.join(os.getcwd(),'CP_Zips.csv')
U_to_Z.to_csv(file,index = False )

In [28]:
Demand_Rate_Zip_Schedule = Demand_Rate_Schedule.merge(U_to_Z[['zip','eiaid']],on ='eiaid',how ='left')
Demand_Rate_Zip_Schedule.sort_values(by =['BA Code','utility','eiaid','name','zip','month','hour'],inplace = True)
Demand_Rate_Zip_Schedule.dropna(inplace = True)
Demand_Rate_Zip_Schedule.reset_index(inplace = True,drop = True)
Demand_Rate_Zip_Schedule.head()

,BA Code,utility,eiaid,name,month,hour,MWh,MWH_M_Scale,schedule,price,zip
0,CPLE,Wake Electric Membership Corp,19981,Large General Coincident Peak Time-of-Use Service,1,0,8257.548387,0.437122,0.0,15.3,27525
1,CPLE,Wake Electric Membership Corp,19981,Large General Coincident Peak Time-of-Use Service,1,1,8304.362903,0.442398,0.0,15.3,27525
2,CPLE,Wake Electric Membership Corp,19981,Large General Coincident Peak Time-of-Use Service,1,2,8268.612903,0.438212,0.0,15.3,27525
3,CPLE,Wake Electric Membership Corp,19981,Large General Coincident Peak Time-of-Use Service,1,3,8074.056452,0.415773,0.0,15.3,27525
4,CPLE,Wake Electric Membership Corp,19981,Large General Coincident Peak Time-of-Use Service,1,4,7752.233871,0.378684,0.0,15.3,27525


In [29]:
len(Demand_Rate_Zip_Schedule.eiaid.unique())

18

In [30]:
Demand_Rate_Zip_Schedule.shape

(1158912, 11)

In [31]:
CP_Zips = Demand_Rate_Zip_Schedule.zip.unique()

# Solar Power

In [34]:
Weather_Files = pd.DataFrame(os.listdir(os.path.join(os.getcwd(),'Weather_Files')),columns = ['Loc'])
Weather_Files = pd.concat([Weather_Files,Weather_Files.Loc.str.split('_',expand = True).rename(columns = {0:'Zip',1:'Lat',2:'Long'})],axis = 1)
Weather_Files_Zips = Weather_Files.Zip
Zip_Daily_Hourly_Solar_Production = pd.DataFrame()

In [35]:
Need_WF_Zips = list(set(CP_Zips) - set(Weather_Files_Zips))
for x in Need_WF_Zips:
    print(x)

In [37]:
for m in CP_Zips:
    print(m)
    Loc = Weather_Files.loc[Weather_Files['Zip'] == m,'Loc'].values[0]
    file = os.path.join(os.getcwd(),'Weather_Files',Loc)
    ssc = PySSC()
    print ('Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2')
    print ('SSC Version = ', ssc.version())
    ssc.module_exec_set_print(0)
    data = ssc.data_create()
    ssc.data_set_string( data, b'solar_resource_file',str.encode(file));  #'b/Users/Avery/Desktop/Coincident_Peak_Project/Weather_Files/17302_39.810052_-76.408374_psmv3_60_tmy.csv' );
    ssc.data_set_number( data, b'system_capacity', 10000 )
    ssc.data_set_number( data, b'module_type', 0 )
    ssc.data_set_number( data, b'dc_ac_ratio', 1.1499999761581421 )
    ssc.data_set_number( data, b'inv_eff', 96 )
    ssc.data_set_number( data, b'losses', 14.075660705566406 )
    ssc.data_set_number( data, b'array_type', 0 )
    ssc.data_set_number( data, b'tilt', 20 )
    ssc.data_set_number( data, b'azimuth', 180 )
    ssc.data_set_number( data, b'gcr', 0.40000000596046448 )
    ssc.data_set_number( data, b'adjust:constant', 0 )
    module = ssc.module_create(b'pvwattsv5')
    ssc.module_exec_set_print( 0 );
    if ssc.module_exec(module, data) == 0:
        print ('pvwattsv5 simulation error')
        idx = 1
        msg = ssc.module_log(module, 0)
        while (msg != None):
            print ('	: ' + msg.decode("utf - 8"))
            msg = ssc.module_log(module, idx)
            idx = idx + 1
        SystemExit( "Simulation Error" );
    ssc.module_free(module)
    annual_energy = ssc.data_get_number(data, b'annual_energy');
    print ('Annual energy (year 1) = ', annual_energy)
    capacity_factor = ssc.data_get_number(data, b'capacity_factor');
    print ('Capacity factor (year 1) = ', capacity_factor)
    kwh_per_kw = ssc.data_get_number(data, b'kwh_per_kw');
    print ('Energy yield (year 1) = ', kwh_per_kw)
    hourly_data = pd.Series(ssc.data_get_array(data,b'gen'))
    hourly_data = pd.concat([Time_DF,hourly_data],axis =  1)
    hourly_data['zip'] = m
    Zip_Daily_Hourly_Solar_Production = pd.concat([Zip_Daily_Hourly_Solar_Production,hourly_data],axis =0)
Zip_Daily_Hourly_Solar_Production.rename(columns = {0:'Production'},inplace = True)

27525
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  13772974.661042437
Capacity factor (year 1) =  15.722573813975387
Energy yield (year 1) =  1377.2974661042438
27544
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14289793.809225364
Capacity factor (year 1) =  16.312550010531236
Energy yield (year 1) =  1428.9793809225362
27557
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  13646599.333838902
Capacity factor (year 1) =  15.578309741825231
Energy yield (year 1) =  1364.6599333838901
27571
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  13947959.135204693
Capacity factor (year 1) =  15.922327779914033
Energy yield (year 1) =  1394.7959135204692
27587
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Pro

47019
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  13212395.317323878
Capacity factor (year 1) =  15.082643056305798
Energy yield (year 1) =  1321.2395317323878
47020
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  13083855.563704714
Capacity factor (year 1) =  14.935908177745109
Energy yield (year 1) =  1308.3855563704715
47021
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  13034902.402669331
Capacity factor (year 1) =  14.880025573823438
Energy yield (year 1) =  1303.4902402669331
47022
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  13007838.06313984
Capacity factor (year 1) =  14.849130209063743
Energy yield (year 1) =  1300.7838063139839
47025
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Proj

Annual energy (year 1) =  13015038.743758112
Capacity factor (year 1) =  14.857350164107434
Energy yield (year 1) =  1301.5038743758112
48837
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  12810507.370334225
Capacity factor (year 1) =  14.623866861112129
Energy yield (year 1) =  1281.0507370334224
48845
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  12999736.587342989
Capacity factor (year 1) =  14.839881949021677
Energy yield (year 1) =  1299.9736587342989
48846
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  12963980.080293704
Capacity factor (year 1) =  14.799064018600117
Energy yield (year 1) =  1296.3980080293702
48849
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  12946853.58190981
Capacity factor (yea

49277
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  12976906.24413541
Capacity factor (year 1) =  14.813819913396587
Energy yield (year 1) =  1297.690624413541
49284
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  13050790.899607055
Capacity factor (year 1) =  14.898163127405315
Energy yield (year 1) =  1305.0790899607055
49285
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  13164396.231206866
Capacity factor (year 1) =  15.027849579003272
Energy yield (year 1) =  1316.4396231206865
49305
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  12727886.97918088
Capacity factor (year 1) =  14.529551346096897
Energy yield (year 1) =  1272.788697918088
49310
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project

Annual energy (year 1) =  12689791.897455428
Capacity factor (year 1) =  14.486063809880626
Energy yield (year 1) =  1268.9791897455427
18934
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  12910737.782163301
Capacity factor (year 1) =  14.738285139455824
Energy yield (year 1) =  1291.07377821633
18936
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  12891329.209677117
Capacity factor (year 1) =  14.71612923479123
Energy yield (year 1) =  1289.1329209677117
18938
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  12796600.110295985
Capacity factor (year 1) =  14.607990993488569
Energy yield (year 1) =  1279.6600110295985
18940
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  12919463.433124946
Capacity factor (year 

...

31539
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  15002046.51727834
Capacity factor (year 1) =  17.12562387817162
Energy yield (year 1) =  1500.2046517278338
31545
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14395996.71737791
Capacity factor (year 1) =  16.43378620705241
Energy yield (year 1) =  1439.5996717377911
31551
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14823597.444901414
Capacity factor (year 1) =  16.92191489143997
Energy yield (year 1) =  1482.3597444901416
31552
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14798887.202056026
Capacity factor (year 1) =  16.89370685166213
Energy yield (year 1) =  1479.8887202056026
31554
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/P

67876
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  16148403.675789583
Capacity factor (year 1) =  18.4342507714493
Energy yield (year 1) =  1614.8403675789584
67882
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  16196996.099482214
Capacity factor (year 1) =  18.48972157475139
Energy yield (year 1) =  1619.6996099482215
37501
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14318789.584441133
Capacity factor (year 1) =  16.34565021054924
Energy yield (year 1) =  1431.8789584441133
37544
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14318789.584441133
Capacity factor (year 1) =  16.34565021054924
Energy yield (year 1) =  1431.8789584441133
38002
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/

38125
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14357958.760020465
Capacity factor (year 1) =  16.3903638813019
Energy yield (year 1) =  1435.7958760020465
38126
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14318789.584441133
Capacity factor (year 1) =  16.34565021054924
Energy yield (year 1) =  1431.8789584441133
38127
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14217710.848732078
Capacity factor (year 1) =  16.230263525949862
Energy yield (year 1) =  1421.7710848732079
38128
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14014873.2173368
Capacity factor (year 1) =  15.998713718421005
Energy yield (year 1) =  1401.48732173368
38130
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/Py

38186
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14318789.584441133
Capacity factor (year 1) =  16.34565021054924
Energy yield (year 1) =  1431.8789584441133
38187
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14318789.584441133
Capacity factor (year 1) =  16.34565021054924
Energy yield (year 1) =  1431.8789584441133
38188
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14318789.584441133
Capacity factor (year 1) =  16.34565021054924
Energy yield (year 1) =  1431.8789584441133
38190
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14318789.584441133
Capacity factor (year 1) =  16.34565021054924
Energy yield (year 1) =  1431.8789584441133
38193
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project

82716
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14504646.711465918
Capacity factor (year 1) =  16.55781588066886
Energy yield (year 1) =  1450.4646711465919
82718
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14537652.392638702
Capacity factor (year 1) =  16.595493598902628
Energy yield (year 1) =  1453.7652392638702
82720
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14430491.464156266
Capacity factor (year 1) =  16.473163771867885
Energy yield (year 1) =  1443.0491464156266
82721
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Project/PySimple2
SSC Version =  208
Annual energy (year 1) =  14472577.066931566
Capacity factor (year 1) =  16.521206697410467
Energy yield (year 1) =  1447.2577066931567
82725
Current folder = /Users/Avery/Desktop/Coincidenet_Peak_Proj

In [38]:
Zip_Hourly_Solar_Production = Zip_Daily_Hourly_Solar_Production.groupby(['zip','month','hour'])['Production'].mean().reset_index()

# Final Merge

In [39]:
Scheduel_Demand_Production_DF = Demand_Rate_Zip_Schedule.merge(Zip_Hourly_Solar_Production,on = ['zip','month','hour'],how = 'left')

In [41]:
Scheduel_Demand_Production_DF.to_csv('Scheduel_demand_Production.csv')